<a href="https://colab.research.google.com/github/bekimpilo/bahbeh/blob/main/Tweepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk


In [ ]:
pip install tweepy

In [ ]:
import re 
import tweepy 
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
nltk.download('punkt')   
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tweepy import OAuthHandler 
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def connect():
  consumer_key = 'KH5kaSlMnY2DpnH87RNPY6yMk'
  consumer_secret = 'RK1IrVvToED7uVoudcGbSpe3PmREY4dD4b6j3VnFOisqqNQWvc'
  access_token = '307810449-8Q7BBn8GlpIFSvzOC0n9fnOKBaDf9PupUpF61v63'
  access_token_secret = 'cEvpS6FL5rIdgNdyWe71y1jeinvxELtcfVBAlbgjOfFye'
  try:
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api
  except:
    print("Error")
    exit(1)

In [ ]:
def cleanText(text):
  text = text.lower()
  # Removes all mentions (@username) from the tweet since it is of no use to us
  text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)
    
  # Removes any link in the text
  text = re.sub('http://\S+|https://\S+', '', text)

  # Only considers the part of the string with char between a to z or digits and whitespace characters
  # Basically removes punctuation
  text = re.sub(r'[^\w\s]', '', text)

  # Removes stop words that have no use in sentiment analysis 
  text_tokens = word_tokenize(text)
  text = [word for word in text_tokens if not word in stopwords.words()]

  text = ' '.join(text)
  return text

In [ ]:
def stem(text):
  # This function is used to stem the given sentence
  porter = PorterStemmer()
  token_words = word_tokenize(text)
  stem_sentence = []
  for word in token_words:
    stem_sentence.append(porter.stem(word))
  return " ".join(stem_sentence)

In [ ]:
from nltk.corpus.reader.plaintext import EuroparlCorpusReader
def sentiment(cleaned_text):
  # Returns the sentiment based on the polarity of the input TextBlob object
  if cleaned_text.sentiment.polarity > 0:
    return 'positive'
  elif cleaned_text.sentiment.polarity < 0:
    return 'negative'
  else:
    return 'neutral'

In [ ]:
def fetch_tweets(query, count = 100000):
  api = connect() # Gets the tweepy API object
  tweets = [] # Empty list that stores all the tweets

  try:
    # Fetches the tweets using the api
    fetched_data = api.search(q = query + ' -filter:retweets', 
count = count)
    for tweet in fetched_data:
      txt = tweet.text
      clean_txt = cleanText(txt) # Cleans the tweet
      stem_txt = TextBlob(stem(clean_txt)) # Stems the tweet
      sent = sentiment(stem_txt) # Gets the sentiment from the tweet
      tweets.append((txt, clean_txt, sent))
    return tweets
  except tweepy.TweepError as e:
    print("Error : " + str(e))
    exit(1)

In [ ]:
tweets = fetch_tweets(query = '#BBMzansi', count = 100000)
# Converting the list into a pandas Dataframe
df = pd.DataFrame(tweets, columns= ['tweets', 'clean_tweets','sentiment'])




In [ ]:
df.head

<bound method NDFrame.head of                                                tweets  ... sentiment
0                           Not Vyno please #BBMzansi  ...   neutral
1   I just love how y'all defend your faves🤣🤣#BBmz...  ...  positive
2   Not her saying Vyno when we have Zino and BU😭 ...  ...   neutral
3   @Lawrence_Maleka please ask Libo what is conte...  ...   neutral
4                          Venus looks cute #BBMzansi  ...  positive
..                                                ...  ...       ...
95  Litetlly everyone thinks Gash nominated them. ...  ...  positive
96  Looks like they are all safe, judging by messa...  ...  positive
97  Finally GORGEOUS hun on the house on the diary...  ...   neutral
98  Ahh you all take this game to heart not everyo...  ...  positive
99                           Venus ♥️♥️♥️🔥  #BBMzansi  ...   neutral

[100 rows x 3 columns]>

In [ ]:
# Dropping the duplicate values just in case there are some tweets that are copied and then stores the data in a csv file
df1 = df.drop_duplicates(subset='clean_tweets')
df1.to_csv('zakesosama.csv', index= False)

In [ ]:
df1

,tweets,clean_tweets,sentiment
0,Libo saying Gash1 sees him as a threat has to ...,libo saying gash1 sees threat funniest thing i...,positive
1,Biggi is the main reason why these housemates ...,biggi main reason housemates way keeps saying ...,negative
2,"Libo, what a gorgeous man. 😍#BBMzansi",libo gorgeous bbmzansi,positive
3,Worse Gash1 approached him twice to talk still...,worse gash1 approached twice talk still nyake ...,positive
4,If Thato leaves I'll really be bored. #BBMzansi,thato leaves really bored bbmzansi,positive
...,...,...,...
95,I really hope Themba will win this season #BBM...,really hope themba win season bbmzansi,positive
96,Libo baby but you're the one that did Gash one...,libo baby youre gash dirty first tho bbmzansi,positive
97,Libo lost his game now he is all about Gash1 i...,libo lost game gash1 ds bbmzansi,negative
98,Here we go again. #BBMzansi https://t.co/cFAyU...,go bbmzansi,positive


In [ ]:
ptweets = df[df['sentiment'] == 'positive']
p_perc = 100 * len(ptweets)/len(tweets)
ntweets = df[df['sentiment'] == 'negative']
n_perc = 100 * len(ntweets)/len(tweets)
print(f'Positive tweets {p_perc} %')
print(f'Neutral tweets {100 - p_perc - n_perc} %')
print(f'Negative tweets {n_perc} %')

Positive tweets 87.0 %
Negative tweets 13.0 %


In [ ]:
len(tweets)

100